In [1]:
import torch
from torch import optim
from tqdm import tqdm

import numpy as np
import copy
import json

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
USE_CUDA = True
use_device = torch.device('cuda' if USE_CUDA else 'cpu')

In [2]:
from data_util import Schema

root = '/notebooks/sinica/dataset/'

schema_root = root+'schema_mr3.txt'
relation_data = root+'skincare_mr3.train'
dev_data = root+'skincare_mr3.dev'
test_data = root+'skincare_mr3.test'

# relation_data = root+'facial_r3.train'
# dev_data = root+'facial_r3.dev'
# test_data = root+'skincare.dev'


schema = Schema(schema_root)
# schema.rel2ix
# {'ApplyIn': 2, 'ApplyTo': 4, 'Feel': 3, 'Rel-None': 1, 'Rel-Pad': 0}

In [3]:
# ========hyper-parameter-set==========
MAX_LEN = 100
BATCH_SIZE = 32

EMBEDDING_DIM = 32
HIDDEN_DIM1 = 20
HIDDEN_DIM2 =  16 #16
LABEL_EMBED_DIM = 8

ATTN_OUT = 8 #8

n_iter = 200
rel_be_filtered=None
# rel_be_filtered=['Feel']

In [4]:
from data_util import BIOLoader

loader = BIOLoader(relation_data, MAX_LEN, BATCH_SIZE, schema, rel_be_filtered=rel_be_filtered, 
                   shuffle=True, device=use_device)
dev_loader = BIOLoader(dev_data, MAX_LEN, BATCH_SIZE, schema, rel_be_filtered=rel_be_filtered, 
                       word_to_ix=loader.word_to_ix, device=use_device)

In [5]:
from models import JointERE


param_list = [loader.vocab_size, EMBEDDING_DIM, HIDDEN_DIM1, HIDDEN_DIM2, LABEL_EMBED_DIM,
             ATTN_OUT, schema]


model = JointERE(*param_list).to(use_device)
model.fit(loader, dev_loader, n_iter=n_iter)


  0%|          | 1/200 [00:15<52:09, 15.73s/it]

epoch: 1 | ent loss 0.7605 | rel loss 0.0192 | total loss 0.7797
         | val ent loss 0.7191 | val rel loss 0.0218


  1%|          | 2/200 [00:31<51:54, 15.73s/it]

epoch: 2 | ent loss 0.8879 | rel loss 0.0253 | total loss 0.9132
         | val ent loss 0.4750 | val rel loss 0.0166


  2%|▏         | 3/200 [00:47<51:49, 15.78s/it]

...

100%|█████████▉| 199/200 [50:39<00:15, 15.27s/it]

epoch: 199 | ent loss 0.0356 | rel loss 0.0035 | total loss 0.0392
           | val ent loss 0.1199 | val rel loss 0.0028


100%|██████████| 200/200 [50:54<00:00, 15.27s/it]

epoch: 200 | ent loss 0.0017 | rel loss 0.0002 | total loss 0.0019
           | val ent loss 0.1126 | val rel loss 0.0030


JointERE(
  (bn): BatchNorm1d(20, eps=1e-05, momentum=0.5, affine=False, track_running_stats=True)
  (word_embeds): Embedding(1700, 32)
  (bilstm): GRU(32, 10, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (top_hidden): LSTMCell(28, 16)
  (hidden2tag): Linear(in_features=16, out_features=13, bias=True)
  (softmax): LogSoftmax()
  (label_embed): Linear(in_features=13, out_features=8, bias=False)
  (attn): Attn(
    (w1): Linear(in_features=24, out_features=8, bias=False)
    (w2): Linear(in_features=24, out_features=8, bias=False)
    (tanh): Tanh()
    (v): Linear(in_features=8, out_features=5, bias=False)
    (softmax): LogSoftmax()
  )
)

In [6]:
torch.save(model.state_dict(), 'relation_extraction_1_new.pth')
# model.load_state_dict(torch.load('relation_extraction_1_new.pth'))

In [7]:
from models import JointERE
e_score, er_score = model.score(dev_loader, rel_detail=True)

['質', '地', '略', '帶', '黏', '稠', ',', '但', '很', '好', '推', '開', ',', '所', '以', '反', '而', '一', '開', '始', '會', '有', '點', '擔', '心', '在', '冷', '冽', '的', '冬', '天', '會', '不', '夠', '滋', '潤', '保', '濕']
['B-SENS', 'I-SENS', 'O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'B-PERC', 'I-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC']
[['Feel-1-A', 'Feel-0-A'], ['Feel-1-A', 'Feel-0-A'], '', '', ['Feel-0-B'], ['Feel-0-B'], '', '', '', ['Feel-1-B'], ['Feel-1-B'], ['Feel-1-B'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], '', '', '', '', '', [], []]

Predict output
['B-SENS', 'I-SENS', 'O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[['Feel-0-A'], ['Feel-0-A'], '', '', ['Feel-0-B'], ['Feel-0-B'], '', '', '', ''

['只', '是', '如', '果', '本', '身', '皮', '膚', '比', '較', '乾', '的', '時', '候', '會', '有', '白', '白', '的', '粉', '卡', '在', '肌', '膚', '乾', '燥', '紋', '路', '裡', ',', '不', '美', '觀', ',', '所', '以', '要', '先', '做', '好', '基', '本', '保', '濕']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-STAT', 'I-STAT', 'I-STAT', 'I-STAT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], [], [], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

Predict output
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-STAT', 'I-STAT', 'I-STAT', 'I-STAT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], [], [],

['瓶', '身', '與', '壓', '頭', '大', '小', '剛', '好', '合', '適', '一', '手', '拿', '握', '.', '好', '拿', '好', '握', '且', '方', '便', '食', '指', '按', '壓', '壓', '頭', '.', '按', '壓', '適', '量', '於', '手', '心', '.', '質', '地', '是', '透', '明', '無', '色', '具', '流', '動', '性', '的', '輕', '油', '水', '狀', '.', '帶', '有', '淡', '淡', '的', '清', '心', '花', '香', '味', '道', '.', '香', '味', '清', '新', '淡', '雅', '.', '令', '人', '有', '舒', '緩', '放', '鬆', '的', '感', '覺', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'B-SENS', 'I-SENS', 'B-PERC', 'I-PERC', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

['因', '我', '冬', '天', '的', '肌', '膚', '非', '常', '的', '乾', '燥', '所', '以', '冬', '天', '使', '用', '保', '濕', '度', '就', '真', '的', '很', '不', '夠']
['O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'B-FUNC', 'I-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', [], [], '', '', '', '', '', '', '', '', '', '', [], [], '', '', [], [], [], '', '', '', '', '', '']

Predict output
['O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'B-FUNC', 'I-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', ['ApplyIn-0-B'], ['ApplyIn-0-B'], '', '', '', '', '', '', '', '', '', '', [], [], '', '', '', '', [], '', '', '', '', '', '']

True
[(2, 3, 3), (14, 15, 3), (18, 20, 1)]
[]
predict
[(2, 3, 3), (14, 15, 3), (18, 20, 1)]
[]
['質', '地', '很', '滑', '潤', '~', '不', '黏', '稠']
['B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'O', 'B-PERC', 'I-PERC']
[[], [], '', '', '', '', '', [], []]

Predict output
[

['至', '於', '夏', '天', '最', '在', '乎', '的', '控', '油', '持', '妝', '度']
['O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O']
['', '', [], [], '', '', '', '', [], [], '', '', '']

Predict output
['O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O']
['', '', ['ApplyIn-0-B'], ['ApplyIn-0-B'], '', '', '', '', ['ApplyIn-0-A'], [], '', '', '']

True
[(2, 3, 3), (8, 9, 1)]
[]
predict
[(2, 3, 3), (8, 9, 1)]
[]
['夏', '天', '到', '來', '控', '油', '和', '持', '久', '度', '是', '最', '重', '要', '的', '事', '情']
['B-SEAS', 'I-SEAS', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[[], [], '', '', [], [], '', '', '', '', '', '', '', '', '', '', '']

Predict output
['B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[[], [], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

True
[(0, 1, 3), (4, 5, 1)]
[]
predict
[(0, 1, 3)]
[]
['用', '了', '2', '週', '週', '左', '右', '後

...

['雖', '然', '質', '地', '很', '清', '爽', ',']
['O', 'O', 'B-SENS', 'I-SENS', 'O', 'B-PERC', 'I-PERC', 'O']
['', '', ['Feel-0-A'], ['Feel-0-A'], '', ['Feel-0-B'], ['Feel-0-B'], '']

Predict output
['O', 'O', 'B-SENS', 'I-SENS', 'O', 'B-PERC', 'I-PERC', 'O']
['', '', ['Feel-0-A'], ['Feel-0-A'], '', ['Feel-0-B'], ['Feel-0-B'], '']

True
[(2, 3, 2), (5, 6, 4)]
[((2, 3, 2), (5, 6, 4), 2)]
predict
[(2, 3, 2), (5, 6, 4)]
[((2, 3, 2), (5, 6, 4), 2)]
['淡', '淡', '的', '香', '味', '很', '舒', '服', '質', '地', '延', '展', '性', '不', '錯', '也', '不', '會', '太', '黏', '膩', '不', '過', '保', '溼', '度', '來', '說', '適', '合', '冬', '天', '的', '早', '上', '保', '養', '晚', '上', '可', '能', '不', '夠', '保', '濕', '用', '了', '一', '個', '月', '覺', '得', '膚', '色', '滿', '明', '亮', '的']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I

['質', '地', ':', '液', '狀']
['B-SENS', 'I-SENS', 'O', 'B-PERC', 'I-PERC']
[['Feel-0-A'], ['Feel-0-A'], '', ['Feel-0-B'], ['Feel-0-B']]

Predict output
['B-SENS', 'I-SENS', 'O', 'B-PERC', 'I-PERC']
[['Feel-0-A'], ['Feel-0-A'], '', ['Feel-0-B'], ['Feel-0-B']]

True
[(0, 1, 2), (3, 4, 4)]
[((0, 1, 2), (3, 4, 4), 2)]
predict
[(0, 1, 2), (3, 4, 4)]
[((0, 1, 2), (3, 4, 4), 2)]
['擦', '在', '臉', '上', '只', '有', '感', '覺', '到', '舒', '服', ',', '真', '的', '是', '一', '款', '春', '夏', '換', '季', '及', '夏', '天', '冷', '氣', '房', '中', '不', '可', '或', '缺', '的', '保', '濕', '產', '品']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['ApplyIn-0-B'], ['ApplyIn-0-B'], '', '', '', '', '', '', '', '', '', ['ApplyIn-0-A'], ['ApplyIn-0-A'], '', '']

Predict output
['O', 'O', 'O', 'O'

['玫', '瑰', '修', '護', '霜', '的', '質', '感', '是', '比', '較', '濃', '稠', '細', '緻', '水', '嫩', '質', '地', ',', '擦', '在', '手', '上', '不', '會', '有', '水', '水', '也', '不', '會', '滴', '落', ',', '味', '道', '很', '清', '爽', '的', '花', '香', '味', ',', '很', '讓', '人', '喜', '歡', ',', '也', '相', '當', '好', '推', '勻', ',', '而', '且', '很', '快', '就', '吸', '收', ',']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PERC', 'I-PERC', 'B-PERC', 'I-PERC', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PERC', 'I-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', '', '', '', '', ['Feel-0-B'], ['Feel-0-B'], [], [], '', '', ['Feel-1-A'], ['Feel-1-A'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['Feel-1-B'], ['Feel-1-B'], ['Feel-1

['雖', '然', '是', '夏', '天', ',', '但', '幾', '乎', '的', '透', '亮', '度', '也', '有', '明', '顯', '的', '感', '覺', '(', '不', '是', '美', '白', '的', '那', '種', '白', '透', ')', ',']
['O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', [], [], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

Predict output
['O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', [], [], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

True
[(3, 4, 3)]
[]
predict
[(3, 4, 3)]
[]
['不', '過', ',', '基', '本', '的', '保', '濕', '度', '是', '有', '的', ',', '可', '能', '比', '較', '適', '合', '我', '夏', '天', '使', '用', '吧']
['O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'I

['味', '道', '很', '天', '然', ';', '無', '添', '加', '的', '很', '清', '新', '自', '然', ',', '聞', '起', '來', '很', '舒', '服', ';', '不', '刺', '鼻']
['B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[['Feel-0-A'], ['Feel-0-A'], '', '', '', '', '', '', '', '', '', ['Feel-0-B'], ['Feel-0-B'], '', '', '', '', '', '', '', '', '', '', '', '', '']

Predict output
['B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[[], [], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

True
[(0, 1, 2), (11, 12, 4)]
[((0, 1, 2), (11, 12, 4), 2)]
predict
[(0, 1, 2)]
[]
['這', '瓶', '凝', '露', '具', '有', '6', '2', '種', '高', '效', '保', '濕', '美', '肌', '成', '分', ',', '是', 'M', 'e', 'd', 'i', 'p', 'l', 'u', 's', '社', '長', '自', '行', '開', '發', '過', '敏', '性', '肌', '膚', '也', '能', '使', '用', '的', '保', '養', '

In [8]:
test_loader = BIOLoader(test_data, MAX_LEN, BATCH_SIZE, schema, rel_be_filtered=rel_be_filtered, 
                        word_to_ix=loader.word_to_ix, device=use_device)

e_score, er_score = model.score(test_loader, silent=True, rel_detail=True)


Entity detection score
precision  	 recall  	 fbeta_score
0.868 		 0.926 		 0.896 	
Entity+Relation detection score 
precision  	 recall  	 fbeta_score  	
0.732 		 0.533 		 0.617 	
confusion matrix 
TP  	 fp  	 tn  	 fn
112 	 41 	 0 	 98 	

Relation type 0
precision  	 recall  	 fbeta_score  	
1.000 		 0.050 		 0.095 	


Relation type 1
precision  	 recall  	 fbeta_score  	
0.731 		 0.333 		 0.458 	


Relation type 2
precision  	 recall  	 fbeta_score  	
0.728 		 0.805 		 0.765 	

--test--
(array([1.        , 0.73076923, 0.728     ]), array([0.05      , 0.33333333, 0.80530973]), array([0.0952381 , 0.45783133, 0.76470588]))



In [9]:
e_score, er_score = model.score(dev_loader, isTrueEnt=True, rel_detail=True)

['質', '地', '略', '帶', '黏', '稠', ',', '但', '很', '好', '推', '開', ',', '所', '以', '反', '而', '一', '開', '始', '會', '有', '點', '擔', '心', '在', '冷', '冽', '的', '冬', '天', '會', '不', '夠', '滋', '潤', '保', '濕']
['B-SENS', 'I-SENS', 'O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'B-PERC', 'I-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC']
[['Feel-1-A', 'Feel-0-A'], ['Feel-1-A', 'Feel-0-A'], '', '', ['Feel-0-B'], ['Feel-0-B'], '', '', '', ['Feel-1-B'], ['Feel-1-B'], ['Feel-1-B'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], '', '', '', '', '', [], []]

Predict output
['B-SENS', 'I-SENS', 'O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[['Feel-0-A'], ['Feel-0-A'], '', '', ['Feel-0-B'], ['Feel-0-B'], '', '', '', ''

['只', '是', '如', '果', '本', '身', '皮', '膚', '比', '較', '乾', '的', '時', '候', '會', '有', '白', '白', '的', '粉', '卡', '在', '肌', '膚', '乾', '燥', '紋', '路', '裡', ',', '不', '美', '觀', ',', '所', '以', '要', '先', '做', '好', '基', '本', '保', '濕']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-STAT', 'I-STAT', 'I-STAT', 'I-STAT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], [], [], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

Predict output
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-STAT', 'I-STAT', 'I-STAT', 'I-STAT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], [], [],

['質', '地', '是', '偏', '稀', '式', '的', '白', '色', '乳', '液', '狀']
['B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PERC', 'I-PERC', 'I-PERC']
[['Feel-0-A'], ['Feel-0-A'], '', '', '', '', '', '', '', ['Feel-0-B'], ['Feel-0-B'], ['Feel-0-B']]

Predict output
['B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PERC', 'I-PERC', 'I-PERC']
[['Feel-0-A'], ['Feel-0-A'], '', '', '', '', '', '', '', ['Feel-0-B'], ['Feel-0-B'], ['Feel-0-B']]

True
[(0, 1, 2), (9, 11, 4)]
[((0, 1, 2), (9, 11, 4), 2)]
predict
[(0, 1, 2), (9, 11, 4)]
[((0, 1, 2), (9, 11, 4), 2)]
['再', '加', '上', '它', '的', '保', '濕', '效', '果', '很', '讓', '我', '滿', '意', ',', '我', '是', '乾', '性', '肌', '膚', ',', '用', '起', '來', '蠻', '夠', '保', '濕', '的', ',']
['O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-STAT', 'I-STAT', 'I-STAT', 'I-STAT', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O']
['', '', '', '', '', ['ApplyTo-0-A'], ['ApplyTo-0-A'], '', '', '', '', '', '', ''

['D', 'R', '.', 'W', 'U', '玻', '尿', '酸', '保', '濕', '精', '華', '霜', '絕', '對', '是', '乾', '妹', '妹', '冬', '天', '一', '定', '要', '入', '手', '的', '好', '貨', '喔', '~', '~']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], '', '', '', '', '', '', '', '', '', '', '']

Predict output
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], '', '', '', '', '', '', '', '', '', '', '']

True
[(19, 20, 3)]
[]
predict
[(19, 20, 3)]
[]
['無', '味', ',', '質', '地', '為', '稍', '濃', '稠', '乳', '霜', '狀', ',', '需', '要', '推', '比', '較', '久', '才', '會', '吸', '收', ',', '且', '吸', '收', '後', '還', '是', '稍', '感', '厚', '重', ',',

['質', '地', '清', '爽', '不', '黏', '膩']
['B-SENS', 'I-SENS', 'B-PERC', 'I-PERC', 'O', 'B-PERC', 'I-PERC']
[['Feel-0-A'], ['Feel-0-A'], ['Feel-0-B'], ['Feel-0-B'], '', [], []]

Predict output
['B-SENS', 'I-SENS', 'B-PERC', 'I-PERC', 'O', 'B-PERC', 'I-PERC']
[['Feel-1-A', 'Feel-0-A'], ['Feel-1-A', 'Feel-0-A'], ['Feel-0-B'], ['Feel-0-B'], '', ['Feel-1-B'], ['Feel-1-B']]

True
[(0, 1, 2), (2, 3, 4), (5, 6, 4)]
[((0, 1, 2), (2, 3, 4), 2)]
predict
[(0, 1, 2), (2, 3, 4), (5, 6, 4)]
[((0, 1, 2), (2, 3, 4), 2), ((0, 1, 2), (5, 6, 4), 2)]
['覺', '得', '是', '很', '適', '合', '油', '性', '肌', '在', '冬', '天', '使', '用', '的', '一', '款', '美', '白', '精', '華', '乳']
['O', 'O', 'O', 'O', 'O', 'O', 'B-STAT', 'I-STAT', 'I-STAT', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O']
['', '', '', '', '', '', ['ApplyTo-0-B'], ['ApplyTo-0-B'], ['ApplyTo-0-B'], '', [], [], '', '', '', '', '', '', '', '', '', '']

Predict output
['O', 'O', 'O', 'O', 'O', 'O', 'B-STAT', 'I-STAT', 'I-STAT', 'O', 'B

...

['基', '本', '上', '如', '果', '不', '在', '乎', '它', '的', '味', '道', '比', '較', '不', '那', '麼', '好', '聞', ',', '是', '可', '以', '持', '續', '使', '用', '的', '產', '品', '.', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', '', '', '', [], [], '', '', '', '', '', [], [], '', '', '', '', '', '', '', '', '', '', '', '', '']

Predict output
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', '', '', '', ['Feel-0-A'], ['Feel-0-A'], '', '', '', '', '', ['Feel-0-B'], ['Feel-0-B'], '', '', '', '', '', '', '', '', '', '', '', '', '']

True
[(10, 11, 2), (17, 18, 4)]
[]
predict
[(10, 11, 2), (17, 18, 4)]
[((10, 11, 2), (17, 18, 4), 2)]
['雖', '然', '質', '地', '很', '清', '爽', ',']
['O', 'O', 'B-SENS'

['氣', '味', '方', '面', ',', '這', '款', '時', '光', '無', '痕', '緊', '緻', '精', '粹', '剛', '擠', '出', '來', '猛', '然', '一', '聞', '我', '以', '為', '是', '甜', '美', '的', '果', '香', ',', '不', '過', '細', '聞', '發', '現', '有', '點', '像', '麝', '香', '之', '類', '比', '較', '強', '的', '氣', '味', ',', '雖', '然', '不', '是', '我', '個', '人', '特', '別', '偏', '好', '的', '味', '道', ',', '不', '過', '並', '不', '是', '嗆', '人', '的', '濃', '香', ',', '在', '塗', '抹', '時', '味', '道', '不', '會', '太', '濃', ',', '所', '以', '還', 'O', 'K', '啦']
['B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'O', 'B-PERC', 'O'

['質', '地', '也', '不', '會', '太', '黏', '.', '.', '.', '至', '少', '出', '門', '不', '會', '沾', '的', '嘴', '唇', '都', '是', '砂', '子', '.', '.', '.']
['B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'B-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[[], [], '', '', '', '', [], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

Predict output
['B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[['Feel-0-A'], ['Feel-0-A'], '', '', '', '', ['Feel-0-B'], [], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

True
[(0, 1, 2), (6, 6, 4)]
[]
predict
[(0, 1, 2), (6, 7, 4)]
[]
['香', '味', '很', '濃', '郁']
['B-SENS', 'I-SENS', 'O', 'B-PERC', 'I-PERC']
[['Feel-0-A'], ['Feel-0-A'], '', ['Feel-0-B'], ['Feel-0-B']]

Predict output
['B-SENS', 'I-SENS', 'O', 'B-PERC', 'I-PERC']
[['Feel-0-A'], ['Feel-0-A'], '',

['不', '過', '對', '於', '冬', '天', '的', '眼', '周', '肌', '膚', '似', '乎', '不', '夠', '保', '濕']
['O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC']
['', '', '', '', [], [], '', '', '', '', '', '', '', '', '', [], []]

Predict output
['O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC']
['', '', '', '', ['ApplyIn-0-B'], ['ApplyIn-0-B'], '', '', '', '', '', '', '', '', '', ['ApplyIn-0-A'], ['ApplyIn-0-A']]

True
[(4, 5, 3), (15, 16, 1)]
[]
predict
[(4, 5, 3), (15, 16, 1)]
[((4, 5, 3), (15, 16, 1), 1)]
['我', '想', '這', '個', '冬', '天', '會', '使', '用', '這', '瓶', '來', '加', '強', '保', '濕', ',', '而', '且', '效', '果', '也', '不', '錯', ',', '也', '會', '買', '一', '瓶', '給', '媽', '媽', ',', '因', '為', '她', '更', '需', '要']
['O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

['保', '濕', '度', '很', '不', '錯', ',', '一', '般', '冬', '天', '我', '都', '要', '加', '乳', '霜', '保', '濕', ',']
['B-FUNC', 'I-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O']
[['ApplyIn-0-A'], ['ApplyIn-0-A'], ['ApplyIn-0-A'], '', '', '', '', '', '', ['ApplyIn-0-B'], ['ApplyIn-0-B'], '', '', '', '', '', '', [], [], '']

Predict output
['B-FUNC', 'I-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[['ApplyIn-0-A'], ['ApplyIn-0-A'], ['ApplyIn-0-A'], '', '', '', '', '', '', ['ApplyIn-0-B'], ['ApplyIn-0-B'], '', '', '', '', '', '', '', '', '']

True
[(0, 2, 1), (9, 10, 3), (17, 18, 1)]
[((0, 2, 1), (9, 10, 3), 1)]
predict
[(0, 2, 1), (9, 10, 3)]
[((0, 2, 1), (9, 10, 3), 1)]
['嫩', '肌', '精', '的', '質', '地', '是', '介', '於', '化', '妝', '水', '跟', '精', '華', '液', '之', '間', ',', '有', '點', '濃', '稠', '又', '不', '會', '太', '濃', '稠']
['O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O',

In [10]:
e_score, er_score = model.score(test_loader, isTrueEnt=True, silent=True, rel_detail=True)


Entity detection score
precision  	 recall  	 fbeta_score
0.870 		 0.925 		 0.897 	
Entity+Relation detection score 
precision  	 recall  	 fbeta_score  	
0.766 		 0.531 		 0.627 	
confusion matrix 
TP  	 fp  	 tn  	 fn
111 	 34 	 0 	 98 	

Relation type 0
precision  	 recall  	 fbeta_score  	
1.000 		 0.051 		 0.098 	


Relation type 1
precision  	 recall  	 fbeta_score  	
0.783 		 0.316 		 0.450 	


Relation type 2
precision  	 recall  	 fbeta_score  	
0.758 		 0.805 		 0.781 	

--test--
(array([1.        , 0.7826087 , 0.75833333]), array([0.05128205, 0.31578947, 0.80530973]), array([0.09756098, 0.45      , 0.78111588]))

